# Flash Attention 2 Modül Testi

In [ ]:
from flash_attn import flash_attn_func
import torch

batch_size = 4        # 4 örnek aynı anda
seqlen = 128          # her örnek 128 token uzunluğunda
hidden_dim = 768      # modelin gömme (embedding) boyutu
head_dim = 64         # her attention head'in boyutu

q = torch.randn(4, 128, 12, 64).half().cuda()
k = torch.randn(4, 128, 4, 64).half().cuda()
v = torch.randn(4, 128, 4, 64).half().cuda()


flash_attn_func(q, k, v)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from flash_attn.modules.mha import MHA  # FlashAttention MHA
import random
torch.manual_seed(42)
random.seed(42)

class FlashAttentionBlock(nn.Module):
    def __init__(self, hidden_dim=768, n_heads=12, dropout=0.0):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_heads = n_heads
        self.head_dim = hidden_dim // n_heads
        
        # LayerNorm (isteğe bağlı)
        self.ln = nn.LayerNorm(hidden_dim)
        
        # FlashAttention MHA; bu modül kendi içinde QKV projeksiyonu yapar.
        self.attn = MHA(
            embed_dim=hidden_dim,
            num_heads=n_heads,
            dropout=dropout,
            causal=True,
            device="cuda",  # ya "cuda" ya da "cpu"
            dtype=torch.float16,  # FlashAttention için FP16 tercih edilir.
            use_flash_attn=True
        )
        
        # Gerekirse, çıktı üzerinde ek bir lineer projeksiyon ekleyebilirsin.
        # Burada çıktı, (total, n_heads, head_dim) şeklinde olacak.
        # Çıkışı hidden_dim'e (yani n_heads * head_dim) birleştirmek için:
        self.out_proj = nn.Linear(hidden_dim, hidden_dim).to(dtype=torch.float16)

    def forward(self, x, attention_mask):
        """
        x: (B, T, hidden_dim)
        attention_mask: (B, T) boolean tensor; 1 gerçek token, 0 pad
        """
        B, T, C = x.size()
        # Önce layer norm
        x = self.ln(x)
        
        # FlashAttention varlen formatı kullanacaksak, giriş x'i (total, hidden_dim) haline getirmeliyiz.
        # Burada total, batch içindeki toplam gerçek token sayısıdır.
        # Eğer padding varsa, attention_mask'ten token sayısını alıp cu_seqlens oluşturacağız.
        seq_lens = attention_mask.sum(dim=1)  # her örnekteki gerçek token sayıları, shape: (B,)
        cu_seqlens = F.pad(seq_lens.cumsum(0), (1, 0), value=0).to(torch.int32)  # shape: (B+1,)
        max_seqlen = seq_lens.max().item()  # batch içindeki en uzun dizi
        
        # x'i "packed" formata getir: (total, hidden_dim)
        x_flat = x.reshape(B * T, C).contiguous()  # Not: Eğer padding varsa, bu tüm T kullanır.
        # Not: Gerçekten istenen, yalnızca gerçek tokenlar (attention_mask==1) olabilir.
        # Ancak, FlashAttention cu_seqlens kullanırken T'nin tüm elemanlarını (padding dahil) alır;
        # cu_seqlens, her örnekteki gerçek token sayısını içerir.
        
        # FlashAttention çağrısı:
        # self.attn; beklediği input x: (total, hidden_dim)
        # Çıkış shape: (total, n_heads, head_dim)
        out = self.attn(x_flat, cu_seqlens=cu_seqlens, max_seqlen=max_seqlen)
        
        # İsteğe bağlı: Çıkışı birleştirip, çıkış projesi uygulamak.
        # Önce, (total, n_heads, head_dim) → (total, hidden_dim)
        out = out.view(B * T, C)
        out = self.out_proj(out)
        
        # Son olarak, tekrar (B, T, hidden_dim) haline getir:
        out = out.view(B, T, C)
        return out

# Parametreler:
B, T, C = 5, 10, 768  # Batch size, sequence length, hidden_dim

# Dummy giriş (random tensor), FP16 olarak üret
x = torch.randn(B, T, C).cuda().half()

# Dummy attention mask: her batch için örnek
# İlk batch: 9 gerçek token, 3 padding; ikinci batch: tamamen dolu (12 gerçek token)
attention_mask = torch.tensor([
    [1]*9 + [0]*3,
    [1]*12
], dtype=torch.bool).cuda()

# Modeli oluştur ve GPU'ya al, FP16 kullan (ağırlıklar da FP16 olacak)
model = FlashAttentionBlock(hidden_dim=C, n_heads=12, dropout=0.0).cuda().half()

# Test et
output = model(x, attention_mask)
print("Output shape:", output.shape)  # Beklenen: (B, T, C)

